In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
if subject_autoencoder_1 == "Denoising_v0": 
    # Assuming 'expressions' is your pandas Series containing the expression data
    expressions_tensor = torch.tensor(expressions.values, dtype=torch.float32).view(-1, 1)  # Reshape to [N, 1]

    # Creating a dataset and dataloader
    dataset = TensorDataset(expressions_tensor, expressions_tensor)  # Input and target are the same for autoencoders
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


    # Instantiate and train with modifications
    model = SimpleAutoencoder(input_dim=1, encoding_dim=16)
    optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=1e-5)  # L2 regularization
    criterion = nn.HuberLoss()

    # Assume dataloader is defined and inputs are properly scaled
    for epoch in range(10):
        for data in dataloader:
            inputs, _ = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

    with torch.no_grad():
        denoised_expressions = model(expressions_tensor).numpy().flatten()

    expressions = denoised_expressions

elif subject_autoencoder_1 == "Free_v0":
    expressions = expressions 

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn as nn

# Step 1: Define the autoencoder architecture
class SimpleAutoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(SimpleAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Step 2: Define the denoising function with versions for different configurations
def denoising_autoencoder(expressions, version):
    # Convert expressions to a PyTorch tensor and reshape for single feature
    expressions_tensor = torch.tensor(expressions.values, dtype=torch.float32).view(-1, 1)
    
    # Create a dataloader to feed data to the model in batches
    dataset = TensorDataset(expressions_tensor, expressions_tensor)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Initialize the autoencoder model
    model = SimpleAutoencoder(input_dim=1, encoding_dim=16)
    
    # Configure the model, optimizer, and loss function based on the version
    if version == "Denoising_v0":
        optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=1e-5)
        criterion = nn.MSELoss()
    elif version == "Denoising_v1":
        optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
        criterion = nn.L1Loss()
    # Add more elif statements for additional versions

    # Step 3: Train the model
    for epoch in range(10):  # Number of epochs
        for data in dataloader:
            inputs, _ = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

    # Step 4: Apply the trained model to denoise the data
    with torch.no_grad():
        denoised_expressions = model(expressions_tensor).numpy().flatten()

    # Return the denoised data and the trained model
    return denoised_expressions, model

# Step 5: Use the function with different configurations
# Example usage:
#denoised_v0, model_v0 = denoising_autoencoder(your_expressions_series, "Denoising_v0")
#denoised_v1, model_v1 = denoising_autoencoder(your_expressions_series, "Denoising_v1")
